# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)

/home/eddyod/programming/pipeline/src


In [2]:
#from library.controller.sql_controller import SqlController
#from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import affine_transform_point
#from library.atlas.brain_structure_manager import BrainStructureManager
#from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
#    read_image, write_image


In [3]:
def get_origins(animal):
    """
    Fetches the origins from disk. The data is already scaled to 10um.
    """

    origins = {}
    dirpath = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/{animal}/origin'
    if not os.path.exists(dirpath):
        return origins
    files = sorted(os.listdir(dirpath))
    for file in files:
        structure = Path(file).stem
        filepath = os.path.join(dirpath, file)
        origin = np.loadtxt(filepath)
        origins[structure] = origin
    return origins

def compute_affine_transformation(src_pts, dst_pts):
    """
    src_pts: Nx3 array of points in brain A
    dst_pts: Nx3 array of points in brain B
    
    Returns:
        T  : 4x4 affine matrix mapping src → dst
        err: RMS error after transform
    """

    src_pts = np.asarray(src_pts)
    dst_pts = np.asarray(dst_pts)
    assert src_pts.shape == dst_pts.shape
    assert src_pts.shape[0] >= 4  # minimum points for 3D affine

    N = src_pts.shape[0]

    # Build homogeneous design matrix: [x y z 1]
    X = np.hstack([src_pts, np.ones((N, 1))])        # Nx4
    Y = dst_pts                                       # Nx3

    # Solve X * A = Y  → A is 4×3 matrix
    A, *_ = np.linalg.lstsq(X, Y, rcond=None)

    # Convert to 4×4 transform
    T = np.eye(4)
    T[:3, :4] = A.T

    # Apply transform to src
    src_h = np.hstack([src_pts, np.ones((N, 1))])    # Nx4
    dst_pred = (T @ src_h.T).T[:, :3]

    # Compute RMS error
    rmse = np.sqrt(np.mean(np.sum((dst_pts - dst_pred)**2, axis=1)))

    return T, rmse, dst_pred


In [13]:
moving_name = 'MD594'
fixed_name = 'AtlasV8'
moving_all = get_origins(moving_name)
fixed_all = get_origins(fixed_name)
common_keys = list(moving_all.keys() & fixed_all.keys())
bad_keys = ('10N_L','10N_R')
good_keys = set(common_keys) - set(bad_keys)
moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
print(len(good_keys))
transformation_matrix, rmse, predictions = compute_affine_transformation(moving_src, fixed_src)
data_path = "/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration"
transformation_matrix_path = os.path.join(data_path, moving_name, f'{moving_name}_{fixed_name}_10.0um')
np.save(transformation_matrix_path, transformation_matrix)
print(transformation_matrix)
print('rmse', rmse)

df_list = []
predicted_dict = {}
for structure in good_keys:    
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    predicted0 = affine_transform_point(moving0, transformation_matrix)
    difference = [a - b for a, b in zip(predicted0, fixed0)]   
    row = [structure, np.round(moving0,2), np.round(fixed0,2), np.round(predicted0,2), np.round(difference,2)]
    df_list.append(row)
    predicted_dict[structure] = predicted0

49
[[ 9.63300395e-01 -1.83586125e-01  5.26920412e-02  3.23984590e+01]
 [ 1.91070126e-01  9.45259787e-01 -2.40011068e-02 -1.60733540e+02]
 [-6.26277652e-02  4.70139342e-02  1.00325498e+00  4.62057591e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
rmse 13.505117209906352


In [5]:
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 DB to Allen RMS: 237.06805950085737 observations: 37
# MD585 to MD594 152.06606097021333 observations: 51
# MD585 to Allen 263.31435 observations: 37
# MD589 to Allen 260.02 observations: 37
# MD594 to Allen 250.79 observations: 37

In [6]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#comtfm = np.array([824.6051918494063, 80.83004570523167, 363.4390121956811])
transformed_structure = affine_transform_point(com, transformation_matrix)

print(f'{moving_name} {structure} non trans {np.round(np.array(com))}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
print(f'{moving_name} {structure} apply trans {np.round(transformed_structure,2)}')
diff = transformed_structure - fixed_all[structure]
#comdiff = comtfm - fixed_all[structure]

print(f'{moving_name}->{fixed_name} error {structure} {diff}')
#print(f'{moving_name}->{fixed_name} tfm error {structure} {comdiff}')

AtlasV8 SC non trans [991. 315. 265.]
Allen SC [821. 101. 365.]
AtlasV8 SC apply trans [823.64 105.44 373.48]
AtlasV8->Allen error SC [2.64317745 4.44089426 8.48385032]


In [7]:
columns = ['structure', moving_name, fixed_name, 'predicted', 'difference']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(2)
df.sort_values(by=['structure'], inplace=True)
#df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)
df.head(50)
#20	3N_R	[1079.0, 531.0, 485.0]	[910.0, 380.0, 17.0]	[873.0, 358.0, 298.0]	[-37.0, -21.0, 281.0]	284.3851
#7	4N_L	[1135.0, 529.0, 423.0]	[959.0, 378.0, 544.0]	[923.0, 362.0, 356.0]	[-36.0, -16.0, -188.0]	192.0327

,structure,AtlasV8,Allen,predicted,difference
Index,,,,,
14,3N_L,"[1072.06, 511.37, 418.67]","[884.0, 366.0, 539.0]","[887.26, 372.9, 533.49]","[3.26, 6.9, -5.51]"
2,3N_R,"[1071.0, 512.57, 447.33]","[884.0, 366.0, 576.0]","[886.52, 376.23, 565.84]","[2.52, 10.23, -10.16]"
13,4N_L,"[1129.38, 519.38, 412.0]","[951.0, 368.0, 534.0]","[945.27, 387.31, 527.22]","[-5.73, 19.31, -6.78]"
15,4N_R,"[1125.15, 519.7, 460.0]","[951.0, 368.0, 586.0]","[941.7, 390.52, 581.44]","[-9.3, 22.52, -4.56]"
29,5N_L,"[1168.76, 594.2, 287.33]","[982.0, 488.0, 382.0]","[975.07, 478.15, 381.0]","[-6.93, -9.85, -1.0]"
20,5N_R,"[1163.29, 584.09, 556.67]","[982.0, 488.0, 700.0]","[975.22, 482.37, 686.85]","[-6.78, -5.63, -13.15]"
24,6N_L,"[1247.9, 616.35, 396.67]","[1063.0, 509.0, 509.0]","[1055.92, 519.98, 505.72]","[-7.08, 10.98, -3.28]"
10,6N_R,"[1247.99, 618.16, 469.33]","[1063.0, 509.0, 593.0]","[1057.04, 527.06, 587.91]","[-5.96, 18.06, -5.09]"
28,7N_L,"[1236.4, 704.91, 293.33]","[1036.0, 639.0, 383.0]","[1032.08, 625.78, 380.78]","[-3.92, -13.22, -2.22]"


In [9]:
registered_origin_path = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/AtlasV8/registered_origin'
for structure, origin in moving_all.items():
    new_origin = affine_transform_point(origin, transformation_matrix)
    new_origin_path = os.path.join(registered_origin_path, f'{structure}.txt')
    print(structure, np.round(origin,2), np.round(new_origin,2))
    np.savetxt(new_origin_path, new_origin)

10N_L [1402.05  623.65  388.  ] [1213.73  541.05  500.56]
10N_R [1398.86  621.44  445.33] [1211.69  541.71  565.6 ]
12N [1386.31  628.01  398.  ] [1197.18  546.04  510.98]
3N_L [1072.06  511.37  418.67] [887.26 372.9  533.49]
3N_R [1071.    512.57  447.33] [886.52 376.23 565.84]
4N_L [1129.38  519.38  412.  ] [945.27 387.31 527.22]
4N_R [1125.15  519.7   460.  ] [941.7  390.52 581.44]
5N_L [1168.76  594.2   287.33] [975.07 478.15 381.  ]
5N_R [1163.29  584.09  556.67] [975.22 482.37 686.85]
6N_L [1247.9   616.35  396.67] [1055.92  519.98  505.72]
6N_R [1247.99  618.16  469.33] [1057.04  527.06  587.91]
7N_L [1236.4   704.91  293.33] [1032.08  625.78  380.78]
7N_R [1239.32  688.59  525.33] [1040.94  620.27  645.13]
7n_L [1185.41  609.97  282.67] [990.32 499.37 374.95]
7n_R [1197.56  604.34  480.  ] [1006.87  506.03  599.44]
AP [1412.54  607.38  414.67] [1226.87  522.79  532.5 ]
Amb_L [1336.52  708.1   318.  ] [1135.28  639.45  411.88]
Amb_R [1340.12  683.99  564.67] [1145.99  624.97  69